<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">

    
# Building a model with local grid refinement

*O.N. Ebbens, Artesia, 2022*

This notebook shows how `nlmod` can be used to create a model with local grid refinement.

In [ ]:
import os

import geopandas as gpd

import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

## Model settings

Modflow 6 makes it possible to use locally refined grids. In `nlmod` you can use a shapefile and a number of levels to specify where and how much you want to use local grid refinement. Below we use a shapefile of the Planetenweg in IJmuiden and set the refinement levels at 2. This well create a grid with cells of 100x100m except at the Planetenweg where the cells will be refined to 25x25m. See figures below.

In [ ]:
# model settings vertex
model_ws = "ijmuiden"
model_name = "IJm_planeten"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
refine_shp_fname = os.path.abspath(os.path.join("data", "planetenweg_ijmuiden"))
levels = 2
extent = [95000.0, 105000.0, 494000.0, 500000.0]
delr = 100.0
delc = 100.0
steady_state = False
steady_start = True
transient_timesteps = 5
perlen = 1.0
start_time = "2015-1-1"
add_northsea = True
starting_head = 1.0

## Download data

First, download the data needed for the model from online sources. It is recommended to store a copy somewhere in case the online dataset changes. For this model we download data from these sources:
- regis (layer model)
- geotop (layer model)
- rijkswaterstaat (surface water)
- jarkus (bathymetry)
- ahn (digital elevation model)
- knmi (precipitation and evaporation)

In [ ]:
# layer models
regis_ds = nlmod.read.regis.download_regis(extent, botm_layer="MSz1",
                                            cachedir=cachedir,
                                            cachename="regis.nc")

geotop_ds = nlmod.read.geotop.download_geotop(extent,
                                              cachedir=cachedir,
                                              cachename="geotop.nc")

In [ ]:
# read surface water data
gdf_surface_water = nlmod.read.rws.get_gdf_surface_water(extent=extent,
                                                         cachedir=cachedir,
                                                         cachename='rws_surface_water.pklz')

# bathymetry
bathymetry_da = nlmod.read.jarkus.download_bathymetry(extent=extent,
                                                      cachedir=cachedir,
                                                      cachename="bathymetry.nc")


In [ ]:
# Digital elevation model
ahn_da = nlmod.read.ahn.download_ahn(extent, cachedir=cachedir, cachename="ahn")

In [ ]:
# knmi data
oc_knmi = nlmod.read.knmi.download_knmi(extent=extent, delr=delr, delc=delc, start='2000-1-1', end='2020-1-1')

## Create model

Modflow 6 makes it possible to use locally refined grids. In `nlmod` you can use a shapefile and a number of levels to specify where and how much you want to use local grid refinement. Below we use a shapefile of the Planetenweg in IJmuiden and set the refinement levels at 2. This well create a grid with cells of 100x100m except at the Planetenweg where the cells will be refined to 25x25m. See figures below.

In [ ]:
# create layer model
layer_model = nlmod.read.regis.add_geotop_to_regis_layers(regis_ds,
                                                          geotop_ds,
                                                          layers="HLc")

# create a model ds by changing grid of layer_model
ds = nlmod.to_model_ds(layer_model, model_name, model_ws, delr=delr, delc=delc)

# add time discretisation
ds = nlmod.time.set_ds_time(
    ds,
    start=start_time,
    steady=steady_state,
    steady_start=steady_start,
    perlen=[perlen] * (transient_timesteps + 1),
)

In [ ]:
ds

## Local grid refinement

The code below applies a local grid refinement to the layer model. The local grid refinement is based on the shapefile 'planetenweg_ijmuiden.shp', which contains a line shape of the Planetenweg, and the levels, which is 2. This means that the model cells at the Planetenweg will get a size of 25 x 25m because we halving the cell size twice (100 / (2^2) = 25). 

In [ ]:
# use gridgen to create vertex grid
ds = nlmod.grid.refine(ds, refinement_features=[(refine_shp_fname, "line", levels)])

# read surface water data
gdf_surface_water = nlmod.read.rws.get_gdf_surface_water(ds=ds,
                                                         cachedir=cachedir,
                                                         cachename='rws_surface_water.pklz')

# add northsea to modelgrid
if add_northsea:
    ds.update(nlmod.read.rws.discretize_northsea(ds, gdf=gdf_surface_water))
    ds.update(nlmod.read.jarkus.discretize_bathymetry(ds, bathymetry_da, cachedir=cachedir, cachename="bathymetry.nc"))
    ds = nlmod.dims.add_bathymetry_to_layer_model(ds)

In [ ]:
# create simulation
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create ims
ims = nlmod.sim.ims(sim)

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# Create discretization
disv = nlmod.gwf.disv(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf, save_flows=True)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=starting_head)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)

In [ ]:
# discretize surface water bodies from geodataframe
da_name = "rws_oppwater"
rws_ds = nlmod.read.rws.discretize_surface_water(
    ds, gdf=gdf_surface_water, da_basename=da_name
)

# add data to model dataset
ds.update(rws_ds)

# build ghb package
ghb = nlmod.gwf.ghb(ds, gwf, bhead=f"{da_name}_stage", cond=f"{da_name}_cond")

# discretize ahn
ahn_ds = nlmod.read.ahn.discretize_ahn(ds, ahn_da)

# add data to model dataset
ds.update(ahn_ds)

# build surface level drain package
drn = nlmod.gwf.surface_drain_from_ds(ds, gwf, resistance=10.0)

# add constant head cells at model boundaries
ds.update(nlmod.grid.mask_model_edge(ds))
chd = nlmod.gwf.chd(ds, gwf, mask="edge_mask", head="starting_head")

In [ ]:
# discretize knmi recharge data
knmi_ds = nlmod.read.knmi.discretize_knmi(ds, oc_knmi, cachedir=cachedir, cachename="recharge")

# update model dataset
ds.update(knmi_ds)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

In [ ]:
ds

## Write and Run

In [ ]:
nlmod.sim.write_and_run(
    sim, ds, write_ds=True, script_path="03_local_grid_refinement.ipynb"
)

## Visualise

Using the `ds` and `gwf` variables it is quite easy to visualise model data. Below the modelgrid together with the surface water is shown.

In [ ]:
plan_weg_gdf = gpd.read_file(refine_shp_fname + ".shp")

# plot modelgrid
ax = nlmod.plot.modelgrid(ds)
nlmod.plot.surface_water(ds, ax=ax)
plan_weg_gdf.plot(ax=ax, color="r", label="Planetenweg")
ax.legend()

# plot zoomed modelgrid
ax = nlmod.plot.modelgrid(ds)
nlmod.plot.surface_water(ds, ax=ax)
ax.set_title("Planetenweg")
plan_weg_gdf.plot(ax=ax, color="r", label="Planetenweg")
ax.set_xlim(100000, 103000)
ax.set_ylim(495000, 497500)
ax.legend();

The model dataset of a vertex model differs from a structured model dataset. The data is stored relative to the cell-id instead of the row and column number. Therefore the model dataset has the dimension icell2d instead of the dimensions x and y. 

In [ ]:
ds

To plot the same rasters as for the previous model we can use the `nlmod.plot.data_array()` function.

In [ ]:
fig, axes = nlmod.plot.get_map(extent, nrows=2, ncols=2, figsize=14)

nlmod.plot.data_array(ds["ahn"], ds, ax=axes[0][0])
nlmod.plot.data_array(ds["botm"][0], ds, ax=axes[0][1])
nlmod.plot.data_array(nlmod.layers.get_idomain(ds)[0], ds, ax=axes[1][0])
nlmod.plot.data_array(ds["edge_mask"][0], ds, ax=axes[1][1])

fig, axes = nlmod.plot.get_map(extent, nrows=2, ncols=2, figsize=(14, 11))
nlmod.plot.data_array(ds["bathymetry"], ds, ax=axes[0][0])
nlmod.plot.data_array(ds["northsea"], ds, ax=axes[0][1])
nlmod.plot.data_array(ds["kh"][1], ds, ax=axes[1][0])
nlmod.plot.data_array(ds["recharge"][0], ds, ax=axes[1][1]);

We can save the entire model as a UGRID NetCDF-file. This can be opened in qgis, as a 'Mesh Layer'. For more information see https://docs.qgis.org/3.16/en/docs/user_manual/working_with_mesh/mesh_properties.html

In [ ]:
fname = os.path.join(ds.figdir, "results.nc")
nlmod.gis.ds_to_ugrid_nc_file(ds, fname)